In [ ]:
pip install ollama

In [ ]:
pip install langchain langchain_community chromadb

Ok, now lets do our RAG

In [3]:
import ollama

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [5]:
from langchain.document_loaders import WebBaseLoader

In [ ]:
#loader = WebBaseLoader("https://tanzil.net/trans/en.shakir")
loader = WebBaseLoader("https://tanzil.net/trans/ur.jawadi")
embeddingModel = 'mxbai-embed-large'
llmModel = 'tadayuki/suzume-llama3'

In [ ]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
splits = text_splitter.split_documents(data)


In [ ]:
print(len(splits))

Now we use mistral to create the embeddings and store them in chroma

In [ ]:
%%time
embeddingModel = 'mxbai-embed-large'
embeddings = OllamaEmbeddings(model=embeddingModel)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [ ]:
%%time
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model=llmModel, messages=[{'role': 'system', 'content': 'You are a Quran assistant who answers the question using the context provided to you'}, {'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


In [ ]:
# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    #print(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [ ]:
%%time
result = rag_chain("اللہ کی صفات کیا ہیں؟?")
print(result)